In [2]:
import os

In [3]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\prac 1\\end-to-prac\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\prac 1\\end-to-prac'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataCleaningConfig:
    root_dir: Path
    cleaned_data: Path
    unzip_data_dir: Path

In [8]:
from engineProject.constants import *
from engineProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        ):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_cleaned_data_config(self) -> DataCleaningConfig:
        config = self.config.data_cleaning

        create_directories([config.root_dir])

        data_cleaning_config = DataCleaningConfig(
            root_dir=config.root_dir,
            cleaned_data=config.cleaned_data,
            unzip_data_dir = config.unzip_data_dir,
        )

        return data_cleaning_config

In [10]:
import os
from engineProject import logger
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler

In [11]:
class DataCleaning:
    def __init__(self, config: DataCleaningConfig):
        self.config = config


    def cleaned_data(self) -> pd.DataFrame:
      
        df = pd.read_csv(self.config.unzip_data_dir)

        
        df= df.sample(frac=1)
        maintenance_df = df.loc[df['Engine Condition'] == 1][:7218]
        great_df = df.loc[df['Engine Condition'] == 0]

        new_df = pd.concat([maintenance_df, great_df])

        new_df = df.sample(frac=1, random_state=42)

        save_path = os.path.join(self.config.root_dir, "new_df.csv")
        new_df.to_csv(save_path, index=False)

        return new_df

In [12]:
try:
    config = ConfigurationManager()
    data_cleaning_config = config.get_cleaned_data_config()
    data_cleaned = DataCleaning(config=data_cleaning_config)
    data_cleaned.cleaned_data()
except Exception as e:
    raise e

[2023-12-19 15:16:58,367: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-19 15:16:58,372: INFO: common: created directory at: artifacts]
[2023-12-19 15:16:58,374: INFO: common: created directory at: artifacts/data_cleaning]


In [13]:
pd.read_csv("artifacts/data_cleaning/new_df.csv")

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,1248,3.212872,8.139221,1.582551,85.037898,86.022991,1
1,494,4.636657,7.889408,2.109195,73.464698,83.551377,1
2,671,1.781894,5.020616,1.721686,76.096380,82.303551,0
3,611,3.141444,8.592298,1.697853,75.976093,73.688796,1
4,529,2.500240,5.339594,6.009598,77.103648,84.223274,0
...,...,...,...,...,...,...,...
19530,731,3.736668,5.555611,2.190859,85.968015,74.572320,0
19531,521,3.937583,6.008644,2.684036,85.487653,71.787794,1
19532,397,5.155546,4.818252,2.075835,76.388027,74.608215,1
19533,588,2.769674,4.308162,3.179618,76.196809,87.962219,1


In [15]:
pd.read_csv("artifacts/data_ingestion/engine_data.csv")

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,700,2.493592,11.790927,3.178981,84.144163,81.632187,1
1,876,2.941606,16.193866,2.464504,77.640934,82.445724,0
2,520,2.961746,6.553147,1.064347,77.752266,79.645777,1
3,473,3.707835,19.510172,3.727455,74.129907,71.774629,1
4,619,5.672919,15.738871,2.052251,78.396989,87.000225,0
...,...,...,...,...,...,...,...
19530,902,4.117296,4.981360,4.346564,75.951627,87.925087,1
19531,694,4.817720,10.866701,6.186689,75.281430,74.928459,1
19532,684,2.673344,4.927376,1.903572,76.844940,86.337345,1
19533,696,3.094163,8.291816,1.221729,77.179693,73.624396,1
